# Repetition + Self Study
**Exercise Session Resource Economics (Spring Term 2025)** \
Raul Hochuli (raul.hochuli@unibas.ch)\ 

--

As discussed in class on Monday I created another notebook for you to do some repetition and self-studying. You will find quiz like "fill in the blanks" questions, which are quite different from the exercises I shared so far. I hope this exercise style will help you learn more efficiently and with less frustraitions. Solutions are also direclty uploaded so you can self-evaluate how well you understood the modeling part of the course so far. **Please don't try to solve this as a mock examn, but rather as an open-book repetition and use all the solution files from the previous exercise sessions**. The goal is that you develop solid understanding of what's happening in a model by reading the code, not that you can memorize code in your head. 

Should you have trouble understanding something after studying the solution file to this notebook, you can of course schedule a 1:1-session with me during office hours where we can close any remaining gaps. 

I hope you will find this notebook useful. 

Best regards and have a nice week
Raul

In [11]:
# Packages used in the notebook
import numpy as np
import pandas as pd
import scipy.optimize as opt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [12]:
# This is a plotting function for easy visual representation of your results. You don't need to understand it to complete the exercises. 
# I added here to keep all "unecessary code" away from the exercises and keep the visualization at the end of each task as short as possible.

def add_trace_1x3_subplot(fig, df, x_col, y_col_tupl, name):
    for i, col in enumerate(y_col_tupl):
        fig.add_trace(go.Scatter(x=df[x_col], 
                                 y=df[y_col_tupl[i]], 
                                 mode='lines', 
                                 name=f'{y_col_tupl[i]}{name}', 
                                 legendgroup=name,
                                 ), row=1, col=i+1)

## 1 Basics (Dataframes and Plots)



You have an array of $x$ and 3 functions $f_i$ defining $y_i$. 
Complete the code below to have a good visualization. 

$$
\begin{array}{l}
x = 0, 1, 2, \dots, 10 \\
\\
y_1 = f_1(x) = x^2 \\
\\
y_2 = f_2(x) = 0.01x^2 - 20x + 100 \\
\\
y_3 = f_3(x) = \cos(x) \cdot 20
\end{array}
$$

Hints for what to do step by step (marked by `<...>` in the code): 

**data frame handling**
1. Define the missing functions $f_i$
2. Use the $y$ arrays created to create a data frame (e.g. column 'x' containing the x values etc.)
3. Print the sum of all columns of your result data frame in one print statement
4. Print the max of all columns of your result data frame


**plotting** 

5. `fig` is an empty plot. Add a traces for every column to visualize the results from your data frame
6. Change the layout of the plot to add titles to the x and y axes and use the pretter `template='plotly_white'`.



In [ ]:

x = np.arange(0, 10 + 1, 1)

def f1(x):
    return x**2

def f2(x):
    return 0.01*x**2 -20*x + 100

def f3(x):
    return np.cos(x) * 20

y1 = f1(x)
y2 = f2(x)
y3 = f3(x)

df = pd.DataFrame({
    'x': x,
    'y1': y1, 
    'y2': y2,
    'y3': y3
})

print(f'sum(x): {df.x.sum()}  sum(y1): {df.y1.sum()}  sum(y2): {df.y2.sum()}  sum(y3): {df.y3.sum()}')  
print(f'max(x): {df.x.max()}  max(y1): {df.y1.max()}  max(y2): {df.y2.max()}  max(y3): {df.y3.max()}')  

fig = go.Figure()
fig.add_trace(go.Scatter(x=df['x'], y=df['y1'], mode='lines+markers', name='y1 = x^2'))
fig.add_trace(go.Scatter(x=df['x'], y=df['y2'], mode='lines+markers', name='y2 = ax^2 + bx + c'))
fig.add_trace(go.Scatter(x=df['x'], y=df['y3'], mode='lines+markers', name='y3 = sin(x)'))
fig.update_layout(title='Graphs of y1, y2 and y3', xaxis_title='x', yaxis_title='y', template='plotly_white')
fig.show()

sum(x): 55  sum(y1): 385  sum(y2): 3.849999999999966  sum(y3): -8.348954929118115


## 2 Optimal Depletion

You have a finite resource oil $R$ which is extracted at $q_t$, with constant costs $c$ for every period $t$ with a constant discount rate $r$. Profit $\pi_t$ is defined below, as well as the maximal extraction constraint. 
Complete the code below so that the optimization model runs with good results.  

$$
\begin{array}{l}
    \pi_t = q_t ^{0.5} - c  \\
    \\
    R_t - q_{t} = R_{t+1}\\
    \\
    \sum_0^{T} q_t \overset{!}{\leq} R_0\\
    \\
    \rho^t = 1/(1+r)^t\\
\end{array}
$$
Hints for what to do step by step (marked by `<...>` in the code): 
1. Define the time varying parameter `rho`
2. Complete the function defining the profit variable `f_profit`
3. Apply the constraint function correctly
4. Run the model again with $R_0 = 100$ and $r = 0.02$. 
5. Add the results of this second model run to the plot. 

In [14]:
def optim_depletion(T = 20, r = 0.07 , c = 1, R0 = 150):
    # params
    rho = 1/(1+r)**np.arange(T+1)

    # State Variables -----------------------------------
    def f_R(i_q):
        R = np.zeros(T+1)
        R[0] = R0
        for t in range(1, T+1):
            R[t] = R[t-1] - i_q[t-1]
        return R

    def f_profit(i_q):
        profit = np.zeros(T+1)
        for t in range(T+1):
            profit[t] = (i_q[t]**0.5 - c)
        return profit

    # Objective Function --------------------------------
    def f_obj(i_q):
        profit = f_profit(i_q)
        obj_value = np.sum(profit * rho)
        return -1* obj_value

    # Decision Variables -------------------------------
    q = np.full(T+1, 2)
    bnds = [(0, R0) for t in range(T+1)]  

    # Constraints --------------------------------------
    def c_total_extr(i_q):
        return R0 - sum([i_q[t] for t in range(T+1)])

    constr = [
        {'type': 'ineq', 'fun': c_total_extr},
    ]


    # Optimization --------------------------------------
    res = opt.minimize(f_obj, q, bounds=bnds, constraints=constr)

    # Results to Dataframe ------------------------------
    q_opt = res.x
    R_opt = f_R(q_opt)

    df = pd.DataFrame({
        't': np.arange(T+1),
        'q': q_opt,
        'R': R_opt,
        'R2': R_opt,
    })
    return df

df_optdepl = optim_depletion()
df_optdepl_R100_r002 = optim_depletion(R0= 100, r = 0.02)


fig = make_subplots(rows=1, cols=3, subplot_titles=('q', 'R', 'profit'))
add_trace_1x3_subplot(fig, df_optdepl, 't', ('q', 'R', 'R2'), '_base')
add_trace_1x3_subplot(fig, df_optdepl_R100_r002, 't', ('q', 'R', 'R2'), '_R100_r002')

fig.update_layout(title='Optimal Depletion', template='plotly_white')
fig.show()


## 3 Hotelling Monopolist

You have a monopolist hotelling model with the characteristics defined below. $S$ defines the finite resource stock, $p_t$ the inverse demand function, $q_t$ the extraction rate and $r$ the constant discount rate.
Complete the code below so that the optimization model runs with good results.
$$
\begin{array}{l}
    S_{t+1} = S_t - q_t \\
    \\
    p_t = a - b q_t\\
    \\
    c_t = c \\
    \\
    \sum_0^{T} q_t \overset{!}{=} S_0\\
    \\
    \rho^t = 1/(1+r)^t\\
\end{array}
$$

Hints for what to do step by step (marked by `<...>` in the code): 
1. Define the function for the state variables $S$
2. Define the constraint function
3. Adjust the extraction costs from a constant $c$ to a linear cost function $c_t = c*q_t$ 
4. (Advanced) Run another model again using the `solver_method='Powell'` and `solver_method='Nelder-Mead`


In [15]:
def hotelling_mono(T = 50, S0 = 100, a=15, b=-0.6, c=0.25, r = 0.05, solver_method = 'SLSQP'):
    rho = 1/(1+r)**np.arange(T+1)

    # State Variables -----------------------------------
    def f_S(i_q):
        S = np.zeros(T+1)
        S[0] = S0
        for t in range(1, T+1):
            S[t] = S[t-1] - i_q[t-1]
        return S
    
    def f_p(i_q):
        return a + b*i_q
    
    def f_c(i_q): 
        return np.full(T+1, c)
    
    # Objective Function -------------------------------
    def f_obj(i_q):
        p = f_p(i_q)
        c = f_c(i_q)

        obj_value = sum(rho[t]*(p[t]-c[t])*i_q[t] for t in range(T+1))

        return -1* obj_value
    
    # Decision Variables -------------------------------
    q = np.full(T+1, 2)
    bnds = [(0, S0) for t in range(T+1)]

    # Constraints --------------------------------------
    def c_total_extr(i_q):
        return S0 - sum([i_q[t] for t in range(T+1)])
    
    cnstr = [
        {'type': 'eq', 'fun': c_total_extr},
    ]   

    # Optimization ---------------------------------
    results = opt.minimize(f_obj, q, bounds=bnds, constraints=cnstr,
                    options={'disp': False}, 
                    method=solver_method
    )

    # Results to Dataframe ------------------------------
    q_opt = results.x
    S_opt = f_S(q_opt)
    p_opt = f_p(q_opt)

    df = pd.DataFrame({
        't': np.arange(T+1),
        'q': q_opt,
        'S': S_opt,
        'p': p_opt,
    })
    return df

df_hotell = hotelling_mono()
df_hotell_powell = hotelling_mono(solver_method='Powell')
df_hotell_nelder = hotelling_mono(solver_method='Nelder-Mead')

fig = make_subplots(rows=1, cols=3, subplot_titles=('q', 'S', 'p'))
add_trace_1x3_subplot(fig, df_hotell, 't', ('q', 'S', 'p'), '_base')
add_trace_1x3_subplot(fig, df_hotell_powell, 't', ('q', 'S', 'p'), '_Powell')
add_trace_1x3_subplot(fig, df_hotell_nelder, 't', ('q', 'S', 'p'), '_Nelder')

fig.update_layout(title='Hotelling Monopoly', template='plotly_white')
fig.show()

        

C:\Users\hocrau00\AppData\Local\Temp\ipykernel_6820\445195663.py:40: RuntimeWarning:

Method Powell cannot handle constraints.

C:\Users\hocrau00\AppData\Local\Temp\ipykernel_6820\445195663.py:40: RuntimeWarning:

Method Nelder-Mead cannot handle constraints.



## 4 Stock Pollution

You have a stock pollution problem identical to the last exercise session. $W$ defines social welfare of a municipality through utility $U$ and damages $D$, $S_t$ the stock of fertilizer emissions in the lake, $e_t$ the level of emissions at every period, $f(S_t)$ the regeneration rate and $r$ the constant discount rate. 
Complete the code below so that the optimization model runs with good results.  

$$
\begin{array}{l}
    W = \sum_{t=0}^{\infty} \rho^t( U(e_t) - D(S_t)) \\
    \\
    S_{t+1} = S_t - f(S_t) + e_t \\
    \\
    f(S_t) = aS_t\\
    \\
    U(e_t) = e^b_t\\
    \\
    D(S_t) = cS_t^d\\
    \\
    \rho_t = \frac{1}{(1+r)^t}\\
\end{array}
$$
Hints for what to do step by step (marked by `<...>` in the code): 

1. Run two model scenarios with `e_bnd_max = 200` and `e_bnd_max = 500` to convince yourself that emission limit has no effect on the "optimal stock pollution level" in the long run. 
2. Run two model scenarios with `S0 = 0` and `S0 = 200` to convince yourself that the inital stock level has no effect on the "optimal stock pollution level" in the long run. 


In [17]:
def stockpollution(T = 50, S0 = 0, a = 0.1, b = 0.6, c = 0.00008, d = 2, e_bnd_max = 200, r = 0.015 ):
    # Sets & Parameters --------------------------------
    rho = 1 / (1 + r) ** np.arange(T + 1)

    # State Variables -----------------------------------
    def f_S(i_e):
        S = np.zeros(T + 1)
        S[0] = S0
        for t in range(1, T + 1):
            S[t] = S[t-1] - a * S[t-1] + i_e[t-1]
        return S

    def f_W(i_e):
        S = f_S(i_e)
        W = np.zeros(T + 1)
        for t in range(T + 1):
            W[t] = rho[t] * ( (i_e[t] ** b) - (c * (S[t] ** d)) )
        return W

    # Objective Function -------------------------------
    def f_obj(i_e):
        W = f_W(i_e)
        return -1 * np.sum(W)

    # Constraints --------------------------------------
    cnstr = []

    # Decision Variables -------------------------------
    e_start = np.full(T+1, S0)
    bnds = [(0, e_bnd_max) for t in range(T+1)]

    # Optimization ---------------------------------
    results = opt.minimize(f_obj, e_start, bounds=bnds, constraints=cnstr, 
                           options={'disp': False})


    # Results to Dataframe ------------------------------
    e_opt = results.x
    S_opt = f_S(e_opt)
    W_opt = f_W(e_opt)

    df = pd.DataFrame({
        't': np.arange(T + 1),
        'e': e_opt,
        'S': S_opt,
        'W': W_opt
    })
    return df

df_base  = stockpollution(e_bnd_max=200)
df_emax500  = stockpollution(e_bnd_max=500)
dfS200 = stockpollution(S0=200)


fig = make_subplots(rows=1, cols=3, subplot_titles=('e', 'S', 'W'))
add_trace_1x3_subplot(fig, df_base, 't', ('e', 'S', 'W'), '_emax200')
add_trace_1x3_subplot(fig, df_emax500, 't', ('e', 'S', 'W'), '_emax500')
add_trace_1x3_subplot(fig, dfS200, 't', ('e', 'S', 'W'), '_S200')

fig.update_layout(title='Hotelling Monopoly', template='plotly_white')
fig.show()
